In [1]:
import pandas
import feather
import datetime
import numpy
from scipy.optimize import curve_fit
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib.colors import Normalize
from matplotlib import ticker
from davitpy.models import *
from davitpy import utils
from aacgmv2 import convert_mlt
%matplotlib inline

In [2]:
# We need to combine the potential model with
# the SAPS location model to limit the data
# to locations where we see SAPS
def saps_pred_func(x, y, dstInp):
#     (a_sx, b_sx, a_sy, b_sy, a_xo, b_xo, a_yo, b_yo, a_o, b_o, theta) = \
#         ( 2.93, 2.67e-3, 2.21, 3e-3, 3.86, 6.03e-2, -0.51, 5.57e-2, 0.985, 0.93e-3, 0.633 )
    a_sx = 3.11
    b_sx = 0.00371
    a_sy = 1.72
    b_sy = 0.000819
    a_xo = 4.59
    b_xo = 0.0633
    a_yo = -1.19
    b_yo = 0.0321
    a_o = 0.893
    b_o = -0.00147
    theta = 0.692
    # parameters from fitting
#     (a_sx, b_sx, a_sy, b_sy, a_xo, b_xo, a_yo, b_yo, a_o, b_o, theta) = \
#         ( 2.58, -0.007, 1.03, -0.023, 3.99,  \
#            0.041, -1.63, 0.02, 1.11, 0.006, 0.68 )
    sigma_x = a_sx + b_sx * dstInp
    sigma_y = a_sy + b_sy * dstInp
    xo = a_xo + b_xo * dstInp
    yo = a_yo + b_yo * dstInp
    amplitude = a_o + b_o * dstInp    
    
    a = (numpy.cos(theta)**2)/(2*sigma_x**2) + (numpy.sin(theta)**2)/(2*sigma_y**2)
    b = -(numpy.sin(2*theta))/(4*sigma_x**2) + (numpy.sin(2*theta))/(4*sigma_y**2)
    c = (numpy.sin(theta)**2)/(2*sigma_x**2) + (numpy.cos(theta)**2)/(2*sigma_y**2)
    outProb = amplitude*numpy.exp( - (a*((x-xo)**2) + 2*b*(x-xo)*(y-yo) 
                            + c*((y-yo)**2)))
    return outProb

### build test dst indices and plot them
sapsPredDF = pandas.DataFrame(columns=["normMLT", "normLAT", "probSAPS", "MLT", "Lat", "dst_index"])
latArr = []
mltArr = []
normLatArr = []
normMltArr = []
probSapsArr = []
dstArr = []
dstIndSel = [ -120., -100., -75., -50, -25., -5. ,0. ]
for z in dstIndSel:
    for x in range( -7, 8 ):
        for y in range( -12, 10 ):        
            normLatArr.append( x )
            normMltArr.append( y )
            dstArr.append( z )
            if y > 0:
                mltArr.append( y )
            else:
                mltArr.append( y + 24 )
            latArr.append( x + 57. )
            probSapsArr.append( saps_pred_func(x,y,z) )
        
sapsPredDF["MLT"] = mltArr
sapsPredDF["Lat"] = latArr
sapsPredDF["normMLT"] = normMltArr
sapsPredDF["normLAT"] = normLatArr
sapsPredDF["probSAPS"] = probSapsArr
sapsPredDF["dst_index"] = dstArr
# Limit SAPS location to where prob > 0.2
sapsPredDF = sapsPredDF[ sapsPredDF["probSAPS"] >= 0.05\
                       ].reset_index(drop=True)
sapsPredDF.head()

,normMLT,normLAT,probSAPS,MLT,Lat,dst_index
0,-6,-7,0.089514,18,50.0,-120.0
1,-5,-7,0.162988,19,50.0,-120.0
2,-4,-7,0.223678,20,50.0,-120.0
3,-3,-7,0.231361,21,50.0,-120.0
4,-2,-7,0.180367,22,50.0,-120.0


In [3]:
# We'll model peak potential as a log function of AsyH index.
def peak_pot_fit_func(asy):
    a_asy, b_asy = (-6.47, 9.48)
    peakPotVal = a_asy + b_asy*numpy.log(asy)    
    return peakPotVal

In [4]:
# To estimate the MLT variations we
# fit second order harmonics!
def mlt_fit_func(mlt, asy):
    c1_a, c1_b, s1_a, s1_b,\
                 c2_a, c2_b, s2_a, s2_b, phiC1_a,\
                 phiC1_b, phiS1_a, phiS1_b = (1.2138, 0.0171, -0.2115,\
                                              -0.0216, 0.7879, -0.0082,\
                                              -0.785, 0.005, 6.5243, 0.9407,\
                                              7.2761, 0.9453)
    
    # Setup the base constants 
    # as functions of Asy index
    c1 = c1_a + c1_b*asy
    s1 = s1_a + s1_b*asy
    c2 = c2_a + c2_b*asy
    s2 = s2_a + s2_b*asy
    phiC1 = phiC1_a + phiC1_b*asy
    phiS1 = phiS1_a + phiS1_b*asy
    # Now get to the actual function
    phiC = (2*numpy.pi/24.) * mlt + phiC1
    phiS = (2*numpy.pi/24.) * mlt + phiS1
    cosTerm = c1 * numpy.cos(phiC)
    sinTerm = s1 * numpy.sin(phiS)
    cos2Term = c2 * numpy.cos(2*phiC)
    sin2Term = s2 * numpy.sin(2*phiS)
    return cosTerm + sinTerm + cos2Term + sin2Term

In [5]:
# To estimate the MLAT variations we
# fit a second order polynomial
def mlat_fit_func(mlat, asy):
    c0_a, c0_b, c1_a,\
            c1_b, c2 = (11.31144, -0.01263, -0.44709,\
         0.00024, 0.0044)
    c0 = c0_a + c0_b * asy
    c1 = c1_a + c1_b * asy
    return c0 + c1*mlat + c2*numpy.square(mlat)

In [6]:
def saps_pot_func( asy, mlt, mlat ):
    phiSaps = peak_pot_fit_func(asy)
    print "phiSaps", phiSaps
    mltSaps = mlt_fit_func(mlt, asy)
    print "mltSaps", mltSaps
    mlatSaps = mlat_fit_func(mlat, asy)
    print "mlatSaps", mlatSaps
    return phiSaps*mltSaps*mlatSaps

In [15]:
print saps_pot_func( 100, 22, 61)
# print mlat_fit_func(60, 120)

phiSaps 37.1870133632
mltSaps 0.696585682603
mlatSaps 0.61235
15.862278325
